In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import time

Strategy: 
- Code to collect movie title and other features into nested list
- Code to automate capture of data from multiple web pages and store in dataframe   

In [14]:
def get_info_v2(soup):
    
    # nested list of films. contains movie title and corresponding features
    movies_list = []
    
    specs = soup.find_all('div',{'class':'lister-item-content'})


    for content in specs:
        
        # contains the information identified in the variables - features of project
        movie_attributes = []

        movie_year = content.find('span',class_='lister-item-year text-muted unbold').text.replace("(", "").replace(")", "") 
        movie_title = content.find_all('a')[0].text

        # movie deets contains many features. Below we get them indiviudally 
        movie_deets = content.find_all('p')[0]
        movie_runtime = movie_deets.find('span', class_='runtime').text
        movie_genre = movie_deets.find('span', class_='genre').text.replace('\n','').strip()
        movie_rating = movie_deets.find('span', class_ = 'certificate').text
        director_stars = content.find_all('p')[2]
        

        # director and gross box office $
        #final_gross = float(votes_gross_split[:-2])
        director = director_stars.find('a').text
        blob = director_stars.findNext('a')
        actor1 = blob.findNext('a').text
        
        votes_gross = content.find('p', class_= "sort-num_votes-visible").text
        votes_gross_split = votes_gross.split('$')[1]
        final_gross = float(votes_gross_split[:-2])
        
        # places variables into movie attribute list
        movie_attributes.append(movie_title)
        movie_attributes.append(movie_runtime)
        movie_attributes.append(movie_genre)
        movie_attributes.append(movie_rating)
        movie_attributes.append(final_gross)
        movie_attributes.append(director)
        movie_attributes.append(actor1)
        movie_attributes.append(movie_year)
        
        # append movie attributes to list 
        movies_list.append(movie_attributes)

    return movies_list

In [15]:
start_pages = [1,51,101,151,201,251,301]

#instantiates empty dataframe
master_df = pd.DataFrame()

master_list = []
for page in start_pages:
    all_url = 'https://www.imdb.com/search/title/?genres=thriller&sort=boxoffice_gross_us,desc&start=' + str(page) +'&explore=title_type,genres&ref_=adv_nxt'
    response = requests.get(all_url)
    page_html = response.text  
    soup = bs(page_html)
    curr_data = get_info_v2(soup)
    curr_movies = pd.DataFrame(curr_data, columns = ['Title', 'Duration', 'Genre','Rating', 'Gross','Director','Lead', 'Year']) 
    
    #concatonates each page into the empty dataframe
    master_df = pd.concat([master_df,curr_movies], axis=0)

In [16]:
master_df

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year
0,The Dark Knight,152 min,"Action, Crime, Drama",PG-13,534.86,Christopher Nolan,Christian Bale,2008
1,The Hunger Games: Catching Fire,146 min,"Action, Adventure, Drama",PG-13,424.67,Francis Lawrence,Jennifer Lawrence,2013
2,The Hunger Games,142 min,"Action, Adventure, Sci-Fi",PG-13,408.01,Gary Ross,Jennifer Lawrence,2012
3,Jurassic Park,127 min,"Action, Adventure, Sci-Fi",PG-13,402.45,Steven Spielberg,Sam Neill,1993
4,Furious 7,137 min,"Action, Crime, Thriller",PG-13,353.01,James Wan,Vin Diesel,2015
...,...,...,...,...,...,...,...,...
45,Now You See Me 2,129 min,"Action, Adventure, Comedy",PG-13,65.08,Jon M. Chu,Jesse Eisenberg,2016
46,Wolf,125 min,"Drama, Horror, Romance",R,65.01,Mike Nichols,Jack Nicholson,1994
47,Friday the 13th,97 min,"Horror, Mystery, Thriller",R,65.00,Marcus Nispel,Jared Padalecki,2009
48,L.A. Confidential,138 min,"Crime, Drama, Mystery",R,64.62,Curtis Hanson,Kevin Spacey,1997


In [17]:
#master_df.to_csv('thrillers_350rows.csv', index=False)